<a href="https://colab.research.google.com/github/Shlok-11/Stock-Price-Predictor/blob/main/stock_price_prediction_using_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt




In [ ]:
!pip install yfinance

In [ ]:
import yfinance as yf # for financial data

In [ ]:
df = yf.download("GOOG", period="5y", interval="1d")
df

In [ ]:
df = df.asfreq('D')
df

In [ ]:
df = df.fillna(method='ffill')
df

In [ ]:
# filter columns
ts = df[['Close']]
ts

In [ ]:
# Plot the data and identify any unusual observations
plt.figure(figsize=(10, 6))
plt.plot(ts.index, ts.values, label='Original Data')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Value')
plt.show()

In [ ]:
# Perform seasonal decomposition
from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = seasonal_decompose(ts, model='additive')

In [ ]:
trend = decomposition.trend
seasonal = decomposition.seasonal
residuals = decomposition.resid

In [ ]:
# Create subplots for each component
fig, axes = plt.subplots(nrows=4, ncols=1, figsize=(10, 8))

# Plot the original time series
axes[0].plot(ts, label='Original')
axes[0].set_ylabel('Original')

# Plot the trend component
axes[1].plot(trend, label='Trend')
axes[1].set_ylabel('Trend')

# Plot the seasonal component
axes[2].plot(seasonal, label='Seasonality')
axes[2].set_ylabel('Seasonality')

# Plot the residuals component
axes[3].plot(residuals, label='Residuals')
axes[3].set_ylabel('Residuals')

# Add titles and legends
axes[0].set_title('Time Series Decomposition')
plt.tight_layout()
plt.show()

In [ ]:
# train/test split
ts_train = ts.iloc[:int(ts.size * .8)]
ts_test = ts.iloc[int(ts.size * .8):]

In [ ]:
# Plot the data and identify any unusual observations
plt.figure(figsize=(10, 6))
plt.plot(ts_train.index, ts_train.values, 'blue', label='Train Data')
plt.plot(ts_test.index, ts_test.values, 'green', label='Test Data')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Value')
plt.show()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
# Normalize the data
scaler = MinMaxScaler()
scaler.fit(ts_train.values) # fit on train
scaled_ts_train_values = scaler.transform(ts_train.values) # transform on train
scaled_ts_test_values = scaler.transform(ts_test.values) # transform on train

In [ ]:
from keras.preprocessing.sequence import TimeseriesGenerator

In [ ]:
n_input = 24
n_features = 1
generator = TimeseriesGenerator(scaled_ts_train_values, scaled_ts_train_values, length=n_input, batch_size=1)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [ ]:
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_input, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [ ]:
model.summary()

In [ ]:
# fit model
model.fit(generator, epochs=50, verbose=0)

In [ ]:
# plot loss
loss_pre_epoch = model.history.history['loss']

plt.plot(range(len(loss_pre_epoch)), loss_pre_epoch)

In [ ]:
test_predictions = []
first_eval_batch = scaled_ts_train_values[-n_input:]

current_batch = first_eval_batch.reshape((1, n_input, n_features))

for i in range(len(ts_test)):

    current_pred = model.predict(current_batch, verbose=0)[0]

    test_predictions.append(current_pred)

    current_batch = np.append(current_batch[:, 1:, :], [[current_pred]], axis=1)

In [ ]:
true_predictions = scaler.inverse_transform(test_predictions)
# true_predictions

In [ ]:
# Plot the data and identify any unusual observations
plt.figure(figsize=(10, 6))
plt.plot(ts_train.index, ts_train.values, 'blue', label='Train Data')
plt.plot(ts_test.index, ts_test.values, 'green', label='Test Data')
plt.plot(ts_test.index, true_predictions, 'orange', label='Forecasted Data')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Value')
plt.show()

In [ ]:
# Plot the original data and forecasted values
plt.figure(figsize=(10, 6))
plt.plot(ts_test.index, ts_test.values, label='Original Data')
plt.plot(ts_test.index, true_predictions, label='Forecasted Data')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Value')
plt.title('ARIMA Forecast')
plt.show()